# Linear Regression

We consider the California Housing dataset which is freely available in **[1]**. 
<br>
<br>
### Demo scenario - Prediction of housing prices for the state of California
* Configuring the **Python machine learning client for SAP HANA (hana-ml)** and connecting to **SAP HANA Cloud**
* Accessing and visually exploring SAP HANA data using **HANA dataframes** in Python
* Training a **regression model** for the housing prices using **hana-ml Predictive Analysis Library (PAL) functions**
* Performing **Model Evaluation** to show the model accuracy obtained
<br>
<br>
#### References
[1] [California Housing dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html)

## Environment setup and preparation

The following cell loads / **imports** diverse modules needed for the current Demo. As you may know a module can be considered to be the same as a **code library**.

### Python packages for SAP HANA

In [1]:
#import hana database client for Python
from hdbcli import dbapi

#import hana-ml package 
import hana_ml
print (hana_ml.__version__)

2.24.25042500


In [2]:
#importing other packages used
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import os

In [3]:
import hana_ml.dataframe as dataframe

hana_address = os.getenv('hana_address', "<your_hana_address>")
hana_port = int(os.getenv('hana_port', 443)) # Use int() for port
hana_user = os.getenv('hana_user', "<your_hana_user>")
hana_password = os.getenv('hana_password', "<your_hana_password>")
hana_encrypt = os.getenv('hana_encrypt', 'True').lower() == 'true' # Handle boolean conversion
hana_schema = os.getenv('HANA_SCHEMA', 'AICOE') # Default schema or your specific one


conn = dataframe.ConnectionContext(address=hana_address,
                                   port=hana_port,
                                   user=hana_user,  
                                   password=hana_password,
                                   encrypt=True, sslValidateCertificate=False,
                                   current_schema=hana_schema)
conn.connection.isconnected()

True

## Data handling using SAP HANA dataframes

The California Housing dataset is loaded into SAP HANA Cloud as indicaded below. It is freely available in [1], and it has also been placed in the current directory. 

The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars. The target variable has been renamed as 'Target' as shown below.

#### References

[1] [California Housing dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html)

In [4]:
df_data = pd.read_csv("./california_housing_dataset.csv", sep = ',')
df_data.head(5)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Target
0,1.2434,52,2.929412,0.917647,396,4.658824,37.80,-122.27,5.00001
1,1.1696,52,2.436000,0.944000,1349,5.396000,37.87,-122.25,5.00001
2,7.8521,52,7.794393,1.051402,517,2.415888,37.86,-122.24,5.00001
3,9.3959,52,7.512097,0.955645,1366,2.754032,37.85,-122.24,5.00001
4,7.8772,52,8.282548,1.049861,947,2.623269,37.83,-122.23,5.00001


The SAP function hana_ml.dataframe.create_dataframe_from_pandas() creates a SAP HANA DataFrame from a Pandas DataFrame and creates a table in SAP HANA, i.e., table_name = 'california_housing'. 

In [5]:
TABLE_NAME = 'CALIFORNIA_HOUSING'
df_remote = dataframe.create_dataframe_from_pandas(connection_context = conn, 
                                                   pandas_df = df_data, 
                                                   table_name = TABLE_NAME,
                                                   force = True,
                                                   replace = False)
display(df_remote.head(5).collect())

100%|██████████| 1/1 [00:00<00:00,  8.02it/s]


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Target
0,1.2434,52,2.929412,0.917647,396,4.658824,37.80,-122.27,5.00001
1,1.1696,52,2.436000,0.944000,1349,5.396000,37.87,-122.25,5.00001
2,7.8521,52,7.794393,1.051402,517,2.415888,37.86,-122.24,5.00001
3,9.3959,52,7.512097,0.955645,1366,2.754032,37.85,-122.24,5.00001
4,7.8772,52,8.282548,1.049861,947,2.623269,37.83,-122.23,5.00001


ConnectionContext.table() returns a DataFrame that represents the specified table, i.e., "california_housing", and its associated schema. Then, we display the SQL query backing the dataframe.

In [6]:
hdf = conn.table(TABLE_NAME, schema=hana_schema)
print(hdf.select_statement)

SELECT * FROM "AICOE"."CALIFORNIA_HOUSING"


In [7]:
hdf.columns

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude',
 'Target']

In [8]:
hdf.dtypes()

[('MedInc', 'DOUBLE', 15, 15, 15, 0),
 ('HouseAge', 'INT', 10, 10, 10, 0),
 ('AveRooms', 'DOUBLE', 15, 15, 15, 0),
 ('AveBedrms', 'DOUBLE', 15, 15, 15, 0),
 ('Population', 'INT', 10, 10, 10, 0),
 ('AveOccup', 'DOUBLE', 15, 15, 15, 0),
 ('Latitude', 'DOUBLE', 15, 15, 15, 0),
 ('Longitude', 'DOUBLE', 15, 15, 15, 0),
 ('Target', 'DOUBLE', 15, 15, 15, 0)]

In [9]:
#Only the use of the collect method, transfers data or result sets from SAP HANA to Python 
print(hdf.head(5).collect())

   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  1.2434        52  2.929412   0.917647         396  4.658824     37.80   
1  1.1696        52  2.436000   0.944000        1349  5.396000     37.87   
2  7.8521        52  7.794393   1.051402         517  2.415888     37.86   
3  9.3959        52  7.512097   0.955645        1366  2.754032     37.85   
4  7.8772        52  8.282548   1.049861         947  2.623269     37.83   

   Longitude   Target  
0    -122.27  5.00001  
1    -122.25  5.00001  
2    -122.24  5.00001  
3    -122.24  5.00001  
4    -122.23  5.00001  


The SAP class method DataFrame.describe() returns a DataFrame that contains various statistics of data attributes.

In [10]:
print(hdf.describe().collect())

       column  count  unique  nulls         mean         std         min  \
0      MedInc   1000     935      0     7.571841    3.355430    0.499900   
1    HouseAge   1000      51      0    34.581000   12.687332    2.000000   
2    AveRooms   1000     993      0     6.702891    4.604450    1.824719   
3   AveBedrms   1000     936      0     1.096594    0.787550    0.500000   
4  Population   1000     764      0  1104.260000  775.770108   13.000000   
5    AveOccup   1000     984      0     2.575271    1.464681    1.304534   
6    Latitude   1000     189      0    35.164800    1.767507   32.680000   
7   Longitude   1000     218      0  -119.623480    1.931527 -122.640000   
8      Target   1000      59      0     4.822741    0.754468    0.806000   

           max      median  25_percent_cont  25_percent_disc  50_percent_cont  \
0    15.000100    7.420200         4.858400         4.848200         7.420200   
1    52.000000   35.000000        25.000000        25.000000        35.000000

### SAP HANA Dataframe filtering

We are able to select rows that match given condition(s).

For instance, the example below selects the subset of rows whose Population is greater than value 300. We then display the first five rows, and confirm the "Population" values are all greater than 300.

In [11]:
hdf_filter=hdf.filter('"Population">300')

print(hdf.select_statement)
print(hdf_filter.select_statement)

SELECT * FROM "AICOE"."CALIFORNIA_HOUSING"
SELECT * FROM (SELECT * FROM "AICOE"."CALIFORNIA_HOUSING") AS "DT_5" WHERE "Population">300


In [12]:
print(hdf_filter.head(5).collect())

   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  1.2434        52  2.929412   0.917647         396  4.658824     37.80   
1  1.1696        52  2.436000   0.944000        1349  5.396000     37.87   
2  7.8521        52  7.794393   1.051402         517  2.415888     37.86   
3  9.3959        52  7.512097   0.955645        1366  2.754032     37.85   
4  7.8772        52  8.282548   1.049861         947  2.623269     37.83   

   Longitude   Target  
0    -122.27  5.00001  
1    -122.25  5.00001  
2    -122.24  5.00001  
3    -122.24  5.00001  
4    -122.23  5.00001  


### Visual data exploration using SAP HANA DataFrame

We use the HANA capabilities **[1]** to visualize a distribution plot for the SAP HANA DataFrame column named "Population".
<br>

#### References

[1] [SAP hana_ml.visualizers.eda - EDAVisualizer.distribution_plot()](https://help.sap.com/doc/1d0ebfe5e8dd44d09606814d83308d4b/2.0.06/en-US/hana_ml.visualizers.html#hana_ml.visualizers.eda.EDAVisualizer.distribution_plot)


In [13]:
#Exploratory Data Visualizations
from hana_ml.visualizers.eda import EDAVisualizer

start = time.time()

eda = EDAVisualizer(enable_plotly=True)
fig, trace, bin_data = eda.distribution_plot(data=hdf, debrief=True, 
                                             column="Population", bins=60, 
                                             x_axis_fontsize=13, x_axis_rotation=1, 
                                             width=600,
                                             title="Distribution of feature: Population", 
                                             height=400)
#fig.show()

end = time.time()
print("Time taken to do this by getting the data from the server was: {}s".format(round(end-start, 3)))

ImportError: cannot import name 'display' from 'IPython.core.display' (/Users/I760054/Documents/programs/Best Practices/sap-btp-ai-best-practices/.venv/lib/python3.12/site-packages/IPython/core/display.py)


Time taken to do this by getting the data from the server was: 1.568s


### Adding an Index to the dataset

In [14]:
hdf_input = hdf.add_id(id_col='ID')
print(hdf_input.head(5).collect())

   ID  MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0   1  1.2434        52  2.929412   0.917647         396  4.658824     37.80   
1   2  1.1696        52  2.436000   0.944000        1349  5.396000     37.87   
2   3  7.8521        52  7.794393   1.051402         517  2.415888     37.86   
3   4  9.3959        52  7.512097   0.955645        1366  2.754032     37.85   
4   5  7.8772        52  8.282548   1.049861         947  2.623269     37.83   

   Longitude   Target  
0    -122.27  5.00001  
1    -122.25  5.00001  
2    -122.24  5.00001  
3    -122.24  5.00001  
4    -122.23  5.00001  


# Train a PAL regression model for house prices

The Python machine learning client for SAP HANA (hana-ml), exposes all Predictive Analysis Library (PAL) functions as well as the Automated Predictive Library (APL) functions in Python for use based on SAP HANA dataframes input data.

### Partitioning the input data

In [15]:
# Partitioning the input data into train, test, validation sub-sets
from hana_ml.algorithms.pal.partition import train_test_val_split

regressdata_hdf=hdf_input.select('ID', 'MedInc', 'HouseAge', 'AveRooms', 
                                 'AveBedrms', 'Population', 'AveOccup', 
                                 'Latitude', 'Longitude', 'Target')

train_hdf, test_hdf, val_hdf = train_test_val_split(data=regressdata_hdf,
                                                    id_column='ID',
                                                    random_seed=2, 
                                                    partition_method='random',
                                                    training_percentage = 0.7,
                                                    testing_percentage = 0.3,
                                                    validation_percentage = 0.0)

print(regressdata_hdf.select_statement)

SELECT "ID", "MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup", "Latitude", "Longitude", "Target" FROM (SELECT CAST(ROW_NUMBER() OVER() AS INTEGER) + 0 AS "ID", * FROM (SELECT * FROM "AICOE"."CALIFORNIA_HOUSING")) AS "DT_15"


### Train a HybridGradientBoostingTree (HGBT) model

The PAL hybrid gradient boosting tree (HGBT) algorithm **[1]**, is a HANA optimized gradient boosting tree implementation supporting mixed feature types (continuous and categorical) as input. It supports **regression** and classification scenarios.

The Hybrid Gradient Boosting model for regression is trained based on the training subset as shown below.
<br>
<br>
#### References

[1] [SAP hana_ml.algorithms.pal package: **HybridGradientBoostingRegressor** algorithm](https://help.sap.com/doc/1d0ebfe5e8dd44d09606814d83308d4b/2.0.07/en-US/pal/algorithms/hana_ml.algorithms.pal.trees.HybridGradientBoostingRegressor.html)

In [16]:
%%time

from hana_ml.algorithms.pal.trees import HybridGradientBoostingRegressor

hgr = HybridGradientBoostingRegressor(
          n_estimators = 20, split_threshold=0.75,
          split_method = 'exact', learning_rate=0.3,
          max_depth=2,
          resampling_method = 'cv', fold_num=5,       
          evaluation_metric = 'rmse', ref_metric=['mae'] )

hgr.fit(train_hdf, features=['ID', 'MedInc', 'HouseAge', 'AveRooms', 
                             'AveBedrms', 'Population', 'AveOccup', 
                             'Latitude', 'Longitude'], label='Target')

CPU times: user 9.74 ms, sys: 5.89 ms, total: 15.6 ms
Wall time: 963 ms


### Feature Importance
Feature importance assigns a score to input features based on their contribution at predicting the response or dependent variable **[1]**.

The higher the score for a feature, the larger influence it has on the model to predict the target variable. The importance scores are in the range of **[0, 1]**. 

We check the model's features importance below. 

For instance, we realise that the most influential feature is **MedInc** with an importance value of **~0.28**. This feature represents the median income in a block group. "A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data" **[2]**.

Moreover, in positions three and five we find features **Longitude** and **Latitude**, respectively. Their respective importance values are **~0.19** and **~0.04**.  
<br>
<br>
#### References

[1] [SAP hana_ml.algorithms.pal package: **HybridGradientBoostingRegressor** algorithm](https://help.sap.com/doc/1d0ebfe5e8dd44d09606814d83308d4b/2.0.07/en-US/pal/algorithms/hana_ml.algorithms.pal.trees.HybridGradientBoostingRegressor.html)

[2] [California Housing dataset description](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset)

In [17]:
hgr.feature_importances_.sort('IMPORTANCE', desc='TRUE').collect()

,VARIABLE_NAME,IMPORTANCE
0,MedInc,0.288127
1,ID,0.287592
2,Longitude,0.194762
3,AveOccup,0.156107
4,Latitude,0.047069
5,AveRooms,0.014984
6,Population,0.011360
7,HouseAge,0.000000
8,AveBedrms,0.000000


### Model Evaluation

The model performance metric (i.e., regression score) used to evaluate our linear regression model is the **coefficient of determination**. It is known as "R squared", and denoted $R^2$ **[1, 2]**.

$R^2$ is basically the proportion of the variation in the response variable that is predictable from the independent variables. That is, the larger $R^2$ is, the more variability is explained by the model.

Typically, $R^2$ varies from 0 to 1; however, there might be cases where negative values can be obtained, refer to **[1]** for further details.

In our particular case, we achieve an $R^2$ score of **~0.77**.
<br>
<br>
#### References

[1] [Coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination)

[2] [Coefficient of Determination (R-Squared)](https://www.mathworks.com/help/stats/coefficient-of-determination-r-squared.html)


In [18]:
#Computes the R2 score

R2=hgr.score( test_hdf,  key='ID', 
          features=['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 
                    'Population', 'AveOccup', 'Latitude', 'Longitude'], 
                    label='Target')
R2

0.7764686923870653

### Improve the HGBT model with optimal parameter search

We decided to search optimal parameter values for our linear regressor. **ParamSearchCV** does exhaustive or random search over specified parameter values with crossover validation (CV) **[1]**
<br>
<br>
#### References

[1] [SAP hana_ml.algorithms.pal package: **ParamSearchCV** algorithm](https://help.sap.com/doc/1d0ebfe5e8dd44d09606814d83308d4b/2.0.07/en-US/pal/algorithms/hana_ml.algorithms.pal.model_selection.ParamSearchCV.html#hana_ml.algorithms.pal.model_selection.ParamSearchCV)


In [19]:
from hana_ml.algorithms.pal.model_selection import ParamSearchCV

hgbr=HybridGradientBoostingRegressor(n_estimators=50, subsample = 0.8, col_subsample_tree=0.7)

ps_hgr3=ParamSearchCV(estimator=hgbr, search_strategy='grid',
                  param_grid={  'learning_rate': [0.05, 0.1, 0.025, 0.04, 0.01],
                                'max_depth': [4, 5, 6, 7, 8, 10],
                                'split_threshold': [0.1, 0.4, 0.7, 1],
                                'min_samples_leaf': [2,3,4,5,6],
                                'col_subsample_split': [0.2,0.4,0.6, 0.8] },
                  train_control={"fold_num": 10, "evaluation_metric": 'rmse'},
                  scoring='mae'
                  )

ps_hgr3.set_scoring_metric('mae')
ps_hgr3.set_resampling_method('cv')
ps_hgr3.fit(data=train_hdf, features=['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 
         label='Target', key='ID')

#### Inspecting Optimized Parameters

The optmized parameters' values are shown below. 

The mapping between the optimized parameter names and the **HybridGradientBoostingRegressor** parameter names are the following:

* MAX_DEPTH = max_depth
* ETA = learning_rate
* COL_SAMPLE_RATE_BYSPLIT = col_subsample_split
* NODE_SIZE = min_samples_leaf
* GAMMA = split_threshold


In [20]:
ps_hgr3.estimator.selected_param_.collect()

,PARAM_NAME,INT_VALUE,DOUBLE_VALUE,STRING_VALUE
0,MAX_DEPTH,4.0,NaN,None
1,ETA,NaN,0.1,None
2,COL_SAMPLE_RATE_BYSPLIT,NaN,0.8,None
3,NODE_SIZE,5.0,NaN,None
4,GAMMA,NaN,0.4,None


In [21]:
# Optimal parameter values selected
hgbt_params = dict(n_estimators = 50, subsample = 0.8, col_subsample_tree=0.7, split_method = 'exact', fold_num=10, 
                  resampling_method = 'cv', evaluation_metric = 'rmse', ref_metric=['mae'],
                  max_depth=10, learning_rate=0.1, col_subsample_split=0.6,  min_samples_leaf=6, split_threshold=0.1)

### Model Re-Training

Model re-training is needed to incorporate the newly optimized algorithm's parameters aiming to maximize model performance; these parameters were obtained in previous section.

In [22]:
%%time
#retrain model with optimal parameters
hgr_optimized = HybridGradientBoostingRegressor(  **hgbt_params )

hgr_optimized.fit(train_hdf, features=['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], label='Target')

CPU times: user 8.33 ms, sys: 4.99 ms, total: 13.3 ms
Wall time: 967 ms


#### Model Re-Trained: Feature Importance
Feature importance assigns a score to input features based on their contribution at predicting the response or dependent variable **[1]**.

The higher the score for a feature, the larger influence it has on the model to predict the target variable. The importance scores are in the range of $[0, 1]$. 

We check the re-trained model's features importance below. 

By comparing against the initial model trained, we observe feature **MedInc** with a decreased importance value. We may remember this feature represents the median income in a block group. "A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data" **[2]**.

Interestingly, feature **Longitude** becomes the most influential feature from position three (initial model trained).

 As a matter of fact, **Longitude** becomes approximately 2 times more influential than its value delivered by the initial model trained.

The enhanced model might be more accurately highlighting geographical locations have stronger influence onto the target variable than thought before. The **target** variable is the **median house value** for California districts **[2]**. 
<br>
<br>
#### References

[1] [SAP hana_ml.algorithms.pal package: **HybridGradientBoostingRegressor** algorithm](https://help.sap.com/doc/1d0ebfe5e8dd44d09606814d83308d4b/2.0.07/en-US/pal/algorithms/hana_ml.algorithms.pal.trees.HybridGradientBoostingRegressor.html)

[2] [California Housing dataset description](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset)

In [23]:
hgr_optimized.feature_importances_.sort('IMPORTANCE', desc='TRUE').collect()

,VARIABLE_NAME,IMPORTANCE
0,Longitude,0.417955
1,MedInc,0.217484
2,AveOccup,0.126965
3,Latitude,0.107347
4,AveRooms,0.070450
5,Population,0.027546
6,HouseAge,0.022832
7,AveBedrms,0.009420


#### Model Re-Trained: Model Evaluation

The model performance metric (i.e., regression score) used to evaluate our linear regression model is the **coefficient of determination**. It is known as "R squared", and denoted $R^2$ **[1, 2]**.

$R^2$ is basically the proportion of the variation in the response variable that is predictable from the independent variables. That is, the larger $R^2$ is, the more variability is explained by the model.

Typically, $R^2$ varies from 0 to 1; however, there might be cases where negative values can be obtained, refer to **[1]** for further details.

In our particular case, the enhanced model achieves an $R^2$ score of **~0.90** from an initial value of **~0.77**. This is an increase of **~0.13**.
<br>
<br>
#### References

[1] [Coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination)

[2] [Coefficient of Determination (R-Squared)](https://www.mathworks.com/help/stats/coefficient-of-determination-r-squared.html)


In [24]:
#Computes the R2 score

R2=hgr_optimized.score( test_hdf,  key='ID', 
          features=['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 
          label='Target')
R2

0.9123927782943512

# Cleanup
Finally, let's remove the table we created in HANA

In [25]:
tables_to_drop = [TABLE_NAME]

for table in tables_to_drop:
    try:
        conn.drop_table(table)
        print(f"Table {table} dropped successfully.")
    except Exception as e:
        print(f"Error dropping table {table}: {e}")

# Close the connection
try:
    conn.close()
    print("Connection closed successfully.")
except Exception as e:
    print(f"Error closing connection: {e}")


Table CALIFORNIA_HOUSING dropped successfully.
Connection closed successfully.


#### Summary

We have achieved the following:

* Configured the **Python machine learning client for SAP HANA (hana-ml)**, and connected to **SAP HANA Cloud**
* Accessed and visually explored SAP HANA data using **SAP HANA DataFrames** in Python
* Trained a **regression model** for the housing prices using **hana-ml Predictive Analysis Library (PAL) functions**
* Performed **Model Evaluation** to show the model accuracy obtained
* Improved the model via optimal parameter search
* Re-trained the model using the newly optimized algorithm's parameters
* Performed **Model Evaluation** of the re-trained model to demonstrate accuracy improvement